# Install

In [ ]:
https://docs.kalshi.com/api-reference/events/get-event?playground=open

https://docs.kalshi.com/api-reference/market/get-trades?playground=open

https://www.youtube.com/watch?v=E2mgWN4ReqQ&list=PLvzuUVysUFOtFsOuzlyynhz8kyRvLF9YD&index=4

https://docs.kalshi.com/api-reference/exchange/get-user-data-timestamp#get-user-data-timestamp



# Embedding Model to Match Markets

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentanceTransformer
pd.options.display.max_columns = 200

# Models to use dunzhang/stella_en_1.5B_v5, dunzhang/stella_en_400M_v5
model = SentanceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()

# Load data
kalshi_markets = pd.read_json("content/all_kalshi_markets.json")
polymarket_markets = pd.read_json("content/all_polymarkets_markets.json")

# create a column for the retrieval
kalshi_markets["bet_description"] = kalshi_markets["title"] + " " + kalshi_markets["subtitle"] + "\n" + kalshi_markets["rules_primary"] + "\nEnd date" + str(kalshi_markets["close_time"])
polymarket_markets["(bet_description)"] = polymarket_markets["question"] + " " + polymarket_markets["description"] + "\nEnd date" + str(polymarket_markets["end_date_iso"])
kalshi_subset = kalshi_markets.dropna(subset=["bet_description"])
polymarket_subset = polymarket_markets.dropna(subset=["bet_description"])
kalshi_subset.drop_duplicates(subset=["event_ticker"], inplace=True)
print("Kalshi markets ", len(kalshi_subset))
print("Polymarket markets ", len(polymarket_subset))




# Fetch Data

### Get Events

In [ ]:
import requests

# r = requests.get("https://trading-api.kalshi.com/trade-api/events")

r = requests.get("https://api.elections.kalshi.com/trade-api/v2/events/")

# querystring = {"with_nested_markets":"true"}

# response = requests.get(url, params=querystring)

response = r.json()

response

In [ ]:
events = response.get("events", [])

for event in events:
    print(f"{event['title']}  —  ({event['category']}, {event['event_ticker']})")

In [11]:
import requests
import json

# Fetch all open events (including nested markets)
r = requests.get(
    "https://api.elections.kalshi.com/trade-api/v2/events?with_nested_markets=True&status=open&limit=200"
)
response = r.json()

# Get all events safely
events = response.get("events", [])

# Loop through each event and print the full JSON
for event in events:
    print("==== Event ====")
    print(json.dumps(event, indent=2))  # Pretty-print full event JSON

    print("==== Markets ====")
    markets = event.get("markets", [])
    for market in markets:
        print(json.dumps(market, indent=2))  # Pretty-print full market JSON
    print("\n")  # Add spacing between events


==== Event ====
{
  "available_on_brokers": false,
  "category": "World",
  "collateral_return_type": "",
  "event_ticker": "KXELONMARS-99",
  "markets": [
    {
      "can_close_early": true,
      "category": "",
      "close_time": "2099-08-01T04:59:00Z",
      "early_close_condition": "This market will close and expire early if the event occurs.",
      "event_ticker": "KXELONMARS-99",
      "expected_expiration_time": "2099-08-01T15:00:00Z",
      "expiration_time": "2099-08-08T15:00:00Z",
      "expiration_value": "",
      "last_price": 8,
      "last_price_dollars": "0.0800",
      "latest_expiration_time": "2099-08-08T15:00:00Z",
      "liquidity": 3503998,
      "liquidity_dollars": "35039.9800",
      "market_type": "binary",
      "no_ask": 92,
      "no_ask_dollars": "0.9200",
      "no_bid": 90,
      "no_bid_dollars": "0.9000",
      "no_sub_title": "Mars",
      "notional_value": 100,
      "notional_value_dollars": "1.0000",
      "open_interest": 13067,
      "open_ti

### Fetch Markets

In [ ]:
!pip install requests_cache tqdm

In [ ]:
from tqdm import tqdm

cursor = ""
total_markets = float('inf')
events = []

with tqdm(total=total_markets, desc="Fetching markets...") as progress_bar:
  while True:
    url = "https://api.elections.kalshi.com/trade-api/v2/events/"

    response = requests.get(url, params=dict(
        limit=200,
        cursor=cursor,
        with_nested_markets=True
    ))

    r = response.json()

    if r.get("cursor") == cursor or not r.get("cursor"):
      break

    events.extend(r.get("events"))

    cursor = r.get("cursor")
    progress_bar.update(len(events))


Fetching markets...: 786259500it [26:02, 1177915.84it/s]

In [ ]:
import pandas as pd

parsed_markets = []
for event in events:
  markets = events.pop("markets", [])
  for m in markets:
    m.update(event)
    parsed_markets.append(m)

  today = pd.to_datetime("today").strftime("%Y-%m-%d")
  kalshi_file = f"/content/data/kalshi_markets_{today}.json"
  with open(kalshi_file, "w") as f:
    json.dump(parsed_markets, f)

  print(len(events))

In [ ]:
kalshi = pd.read_json(kalshi_file)
kalshi

### Listen over Websockets (live trade)

In [ ]:
import asyncio
import websockets
import json
import requests

KALSHI_WEBSOCKET = "wss://api.elections.kalshi.com/market_lifecycle_v2"

data = {"email": "youraccount", "password": "yourpass"}

r = requests.post("https://api.elections.kalshi.com/login", json=data)
response = r.json()

print(response)

In [ ]:
headers = {"Authorization": f"Bearer {response['token']}"}

trade_message = '{"id": 1, "cmd": "subscribe", "params": {"channels": ["trade"]}}'

async def process_message(message):
  print(f"Recieved message: {message}")

async def listen_to_server():
  async with websockets.connect(KALSHI_WEBSOCKET, extra_headers=headers) as websocket:
    await websocket.send(trade_message)
    print(f"Sent trade message: {trade_message}")
